In [1]:
import pandas as pd

# Читаем данные
df = pd.read_csv('../data/raw/heart.csv')

# Быстрый осмотр
print("Размерность:", df.shape)
df.head()


Размерность: (303, 14)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Проверка и удаление пропусков и дубликатов

In [ ]:
# 1) Смотрим информацию по типам и пропускам
print(df.info())
print("\nПропуски в каждом столбце:")
print(df.isnull().sum())

# 2) Проверяем дубликаты
n_dup = df.duplicated().sum()
print(f"\nНайдено дубликатов: {n_dup}")

# 3) Если есть - удаляем
if n_dup > 0:
    df = df.drop_duplicates()
    print(f"Новая размерность после удаления дубликатов: {df.shape}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None

Пропуски в каждом столбце:
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
outp

Проверка и фильтрация аномалий

In [3]:
# Проверяем диапазоны
for col, (low, high) in {
    'age': (0, 120),
    'trtbps': (50, 250),        # кровяное давление
    'chol': (100, 600),         # уровень холестерина
    'thalachh': (50, 250),      # максимальная ЧСС
    'oldpeak': (0.0, 10.0),     # депрессия ST-сегмента
}.items():
    invalid = df[(df[col] < low) | (df[col] > high)]
    print(f"{col}: найдено аномальных {len(invalid)} записей")
    # если есть - удаляем
    if not invalid.empty:
        df = df[(df[col] >= low) & (df[col] <= high)]
        print(f"  → после фильтрации: {df.shape[0]} строк")


age: найдено аномальных 0 записей
trtbps: найдено аномальных 0 записей
chol: найдено аномальных 0 записей
thalachh: найдено аномальных 0 записей
oldpeak: найдено аномальных 0 записей


Dummy-кодирование категориальных признаков

In [4]:
# Список категориальных столбцов
cat_cols = [
    'sex',    # 0=женщина, 1=мужчина
    'cp',     # тип боли в груди 1-4
    'fbs',    # >120 mg/dl
    'restecg',# результаты ЭКГ 0-2
    'exng',   # стенокардия при нагрузке
    'slp',    # наклон ST 1-3
    'caa',    # число крупных сосудов 0-3
    'thall',  # 3,6,7
]

# Преобразуем
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
print("Новая форма:", df.shape)


Новая форма: (302, 23)


Сохранение очищенных данных

In [5]:
df.to_csv('../data/processed/heart_preprocessed.csv', index=False)
print("Сохранено в data/processed/heart_preprocessed.csv")

Сохранено в data/processed/heart_preprocessed.csv
